# NGC 3568 Photometry Checker

This notebook is designed to check the photometry between the FLCs, Hubble Pipeline DRCs, and my DRCs.

## Imports

In [ ]:
# Python Imports
from os import path
from glob import iglob

# 3rd Party Imports
import numpy as np

# Astropy Imports
from astropy import units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import photutils as phot

## Notebook Setup

In [ ]:
# Paths
TOP_LVL_DIR = '/home/wwaldron/Research/Galaxies/NGC3568/Images'
MAST_DIR    = path.join(TOP_LVL_DIR, 'mastDownload/HST')
PROC_DIR    = path.join(TOP_LVL_DIR, 'ProcessedImages/HST')

## Load Data

In [ ]:
# Load HST DRCs
hstData = {}
for fn in iglob(path.join(MAST_DIR, '**/*drc.fits')):
    
    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']
    
        # Store the Name using the filter as the dict key
        # Start the Empty List if Key does not exist
        if filt not in hstData:
            hstData[filt] = {}
    
        # Load the Image Data and WCS
        hstData[filt]['img'] = hduList[1].data
        hstData[filt]['wcs'] = WCS(hduList[1])

In [ ]:
# Load My DRCs
myData = {}
for fn in iglob(path.join(PROC_DIR, '**/*drc.fits.gz')):
    
    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']
    
        # Store the Name using the filter as the dict key
        # Start the Empty List if Key does not exist
        if filt not in myData:
            myData[filt] = {}
    
        # Load the Image Data and WCS
        myData[filt]['img'] = hduList[1].data
        myData[filt]['wcs'] = WCS(hduList[1])

## Photometry

In [ ]:
# Establish Source Coordinates
srcCrds = SkyCoord(
    ra=[
        167.6867031,
        167.6969115,
        167.6788570,
        167.6835468,
    ],
    dec=[
        -37.4299580,
        -37.4374874,
        -37.4398610,
        -37.4636773,
    ],
    unit=u.deg,
    frame='icrs'
)

# Radii
R_IN  = 0.5 * u.arcsec
R_OUT = 0.9 * u.arcsec

# Area
A_IN  = np.pi*R_IN**2
A_OUT = np.pi*(R_OUT**2 - R_IN**2)

# Setup Apertures
aper = phot.aperture.SkyCircularAperture(srcCrds, R_IN)
annu = phot.aperture.SkyCircularAnnulus(srcCrds, R_IN, R_OUT)

### Simple Aperture Photometry

In [ ]:
# Loop through HST Data
for fn, dataDict in hstData.items():
    
    # Get Photometry
    aperRes = phot.aperture.aperture_photometry(
        dataDict['img'],
        aper,
        wcs=dataDict['wcs']
    )
    annuRes = phot.aperture.aperture_photometry(
        dataDict['img'],
        annu,
        wcs=dataDict['wcs']
    )
    
    # Store Results
    dataDict['flux'] = aperRes['aperture_sum'] - annuRes['aperture_sum']*(A_IN/A_OUT)

In [ ]:
# Loop through My Data
for fn, dataDict in myData.items():
    
    # Get Photometry
    aperRes = phot.aperture.aperture_photometry(
        dataDict['img'],
        aper,
        wcs=dataDict['wcs']
    )
    annuRes = phot.aperture.aperture_photometry(
        dataDict['img'],
        annu,
        wcs=dataDict['wcs']
    )
    
    # Store Results
    dataDict['flux'] = aperRes['aperture_sum'] - annuRes['aperture_sum']*(A_IN/A_OUT)